In [ ]:
import re
import os
import pickle
import numpy as np
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate
from data.configs.demo import config
from torch.utils.data import DataLoader

In [ ]:
from utils.MIND import MIND_bert

In [ ]:
train_path = "C:/MIND/MINDdemo_train/"
config.embedding = 'bert'
config.bert = 'bert-base-uncased'
config.signal_length = 102
config.his_size = 54
a = MIND_bert(config, train_path + 'news.tsv', train_path + 'behaviors.tsv')

In [ ]:
a[10]['his_encoded_index']

In [ ]:
loader_train = DataLoader(a, batch_size=config.batch_size, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, collate_fn=my_collate)

In [ ]:
next(iter(loader_train))